In [ ]:
import torch
print(torch.__version__)

In [ ]:
import cv2
from torchvision import models
import torch
import numpy as np

# category of object should be detected here is the list from documentaion itself
cats = [
    '__background__', 'person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus',
    'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'N/A', 'stop sign',
    'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow',
    'elephant', 'bear', 'zebra', 'giraffe', 'N/A', 'backpack', 'umbrella', 'N/A', 'N/A',
    'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball',
    'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket',
    'bottle', 'N/A', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl',
    'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza',
    'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'N/A', 'dining table',
    'N/A', 'N/A', 'toilet', 'N/A', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone',
    'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'N/A', 'book',
    'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush'
]


#### Defining a function to read the video frame #####
def read_img(img):
    img=cv2.cvtColor(img,cv2.COLOR_BGR2RGB) #After getting frame coverting frame to "BGR-->RGB"
    return img

def to_tensor(img):
    img=torch.tensor(img,dtype=torch.float) # made tensor
    img=img.permute(2,0,1)
    img=img/img.max()
    img=img.unsqueeze(axis=0)
    return img

def get_plot(preds,thresh,img):      # Made plot per frame getting feed rectangle shape 
    idx=(preds[0]['scores']>thresh).cpu().numpy()
    boxes=preds[0]['boxes'].detach().cpu().numpy()[idx]
    labels=preds[0]['labels'].cpu().numpy()[idx]
    for box,label in zip(boxes,labels):
        xmin,ymin,xmax,ymax=[int(x) for x in box]
        img=cv2.rectangle(img,(xmin,ymin),(xmax,ymax),color=(255,0,0), thickness=2)
        font=cv2.FONT_HERSHEY_SIMPLEX
        img=cv2.putText(img,cats[label],(xmin-1,ymax-1),fontFace=font,\
                        color=(0,255,255),fontScale=0.8,lineType=cv2.LINE_AA,thickness=2)
    return img



if __name__=='__main__':
    device=torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    frcnn=models.detection.fasterrcnn_resnet50_fpn(pretrained=True)   # Taken pretrained model"fasterrcnn_resnet50_fpn"
    frcnn.eval()
    frcnn=frcnn.to(device)
    path=r'traffic.mp4'    #here is path for video file
    cap=cv2.VideoCapture(path)
    while True:
        
        ret,frame=cap.read() 
        img=read_img(frame)   #taking frame 
        img_tensor=to_tensor(img)       #made tensor for each frame
        img_tensor=img_tensor.to(device)  
        preds=frcnn(img_tensor)
        im=get_plot(preds,0.50,img)
        
        cv2.imshow("frame",im)
        k=cv2.waitKey(1)
        if k==-1:
            continue
        else:
            break
                
    cap.release()
    cv2.destroyAllWindows() 
        